In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.utils import to_categorical

## Load and Preprocess the Text

In [2]:
# Sample text (you can replace this with a larger dataset)
text = """
Long Short-Term Memory (LSTM) networks, a variant of Recurrent Neural Networks (RNNs), are particularly effective for next word prediction tasks due to their ability to retain long-term dependencies in sequential data. These networks solve the vanishing gradient problem that limits traditional RNNs by introducing a memory cell and gated mechanisms to control the flow of information.
For next word prediction, an LSTM processes input sequences, such as a sentence fragment, to predict the most probable word that follows. The model starts by tokenizing and encoding the text into numerical sequences, ensuring that each word is represented as a unique token. During training, sliding windows of word sequences are created to generate input-output pairs, where the input is a sequence of preceding words and the output is the next word.
After passing through an embedding layer for dense word vector representations, the LSTM layer captures temporal dependencies in the input sequence. A final dense layer with a softmax activation predicts the probability of each word in the vocabulary as the next word. The model learns contextual relationships in text, enabling it to generate coherent and meaningful predictions. For example, given the input "The weather is very," the model might predict "sunny" as the next word.
This approach can be extended with techniques like bidirectional LSTMs, beam search, or temperature sampling to improve context understanding and diversity in predictions. Such models find applications in auto-completion, chatbots, and language modeling, making them essential in natural language processing tasks.
"""

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
word_index = tokenizer.word_index  # Dictionary of word -> index
vocab_size = len(word_index) + 1   # Vocabulary size (+1 for padding)

# Convert text to sequences of integers
sequences = []
for sentence in text.split("."):
    tokens = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokens)):
        # Create input-output pairs
        sequences.append(tokens[:i+1])

# Pad sequences to ensure uniform length
max_len = max([len(x) for x in sequences])


padded_input_sequences = pad_sequences(sequences, maxlen = max_len, padding='pre')

# Split into inputs (X) and outputs (y)
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:, -1]

# Convert outputs to one-hot encoding
y = to_categorical(y, num_classes=vocab_size)

In [3]:
# Reshape X to have the required dimensions (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

In [4]:
X.shape

(238, 32, 1)

In [5]:
y.shape

(238, 147)

In [6]:
vocab_size

147

In [7]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (238, 32, 1)
Shape of y: (238, 147)


In [8]:
# Remove the extra dimension from X
X = X.squeeze()  # Shape becomes (238, 32)

In [9]:
X.shape

(238, 32)

## Build the LSTM Model

In [10]:
# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=X.shape[1]),  # Embedding layer
    GRU(150, return_sequences=True),  # First LSTM, returns sequences
    GRU(150),                         # Second LSTM, returns the final state
    Dense(vocab_size, activation='softmax')  # Output layer
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## train model

In [13]:
# Train the model
model.fit(X, y, epochs=100, batch_size=32)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 97ms/step - accuracy: 0.0124 - loss: 4.9899
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.0593 - loss: 4.9499
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.0589 - loss: 4.8257
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 200ms/step - accuracy: 0.0770 - loss: 4.5852
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 283ms/step - accuracy: 0.0791 - loss: 4.6197
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - accuracy: 0.1003 - loss: 4.4117
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.0711 - loss: 4.4888
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.0710 - loss: 4.3301
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.0584 - loss: 4.2675
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 99ms/step - accuracy: 0.1073 - loss: 4.0773
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.0647 - loss: 3.9832
Epoch 12/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.0

## Predict the Next Word

In [14]:
import time
text = "After passing through an embedding"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
After passing through an embedding layer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
After passing through an embedding layer for
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
After passing through an embedding layer for dense
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
After passing through an embedding layer for dense word
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
After passing through an embedding layer for dense word vector
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
After passing through an embedding layer for dense word vector representations
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
After passing through an embedding layer for dense word vector representations the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
After passing through an embedding layer for dense word vector representations the lstm
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
After passing through an embedding layer for dense word vector representations the lstm layer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
After passing throu